# Test barrier mode

Tenemos que asegurarnos de que tenemos el número de hebras necesario para acometer todas las tareas en paralelo

In [ ]:
sc.stop()

sc = SparkContext(master='local[4]')

In [ ]:
sc.getConf().get('spark.master')

Crea un RDD distribuido en varias particiones

In [ ]:
rdd = sc.range(10000, numSlices=4)

In [ ]:
rdd.getNumPartitions()

Crea la barrera asociada al RDD

In [ ]:
br = rdd.barrier()

La tarea que será ejecutada en cada partición

In [ ]:
from pyspark import BarrierTaskContext

def task(it):
    # obtiene el contexo de la partición
    ctx = BarrierTaskContext.get()
    # realiza la tarea. En este caso, simplemente suma los datos
    result = sum(it)
    # se asegura de que todas las particiones están sincronizadas
    ctx.barrier()
    # devuelve el resultado
    yield result


Lanza las tareas

In [ ]:
c1 = br.mapPartitions(task)

In [ ]:
c2 = c1.map( lambda v: v+1 )

Obtiene el resultado

In [ ]:
c2.collect()